Loading the dataset


In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
! gdown --id 1E8OTtNK_UNSAdmkg3OuB0k2FsKKF9QpY
! gdown --id 1-pm6_kvT9CBrYER40uIkRiG0KMlHnH7L
! gdown --id 1oiVQignVg1wvE1LXKFvOLHPl0P8GNSDp

! gdown --id 1pGkEyDelD7Lp7yhFML9Dt1oo-91Mqf9H
! gdown --id 1DYVzeYjjTTeHtazHfKD9FTx57h-77bkC

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1E8OTtNK_UNSAdmkg3OuB0k2FsKKF9QpY
To: /content/train.csv
100% 98.6M/98.6M [00:01<00:00, 88.1MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-pm6_kvT9CBrYER40uIkRiG0KMlHnH7L
To: /content/test.csv
100% 25.1M/25.1M [00:00<00:00, 70.6MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
submit_data = pd.read_csv("submit.csv")

train_data.dropna(inplace=True)
train_data.drop(train_data.loc[train_data['text']==" "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="  "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="\n"].index, inplace=True)

# Resetting index
train_data.reset_index(inplace=True)
train_data= train_data.apply(lambda x: x.astype(str).str.lower())
train_data = train_data.drop(columns=["id", "index"])

test_data = test_data.join(submit_data["label"])
transformed_df = pd.read_csv("pre_train.csv")
transformed_df_test = pd.read_csv("pre_test.csv")

# X_test = transformed_df_test.values
y_test = test_data["label"].values 


In [ ]:
# Checking null values
test_data.isnull().sum()

id          0
title     122
author    503
text        7
label       0
dtype: int64

In [ ]:
# Removing null values
test_data.dropna(inplace=True)
test_data.drop(test_data.loc[test_data['text']==" "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="  "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="\n"].index, inplace=True)

# Resetting index
test_data.reset_index(inplace=True)

In [ ]:
test_data= test_data.apply(lambda x: x.astype(str).str.lower())

In [ ]:
test_data = test_data.drop(columns=["id","index"])
y_test = test_data["label"].values 

In [ ]:
stop_words_set = set(stopwords.words('english'))

sw_removed = []

for i in range(len(train_data)):
    review = re.sub('[^a-zA-Z]',' ',train_data['text'][i])
    #not on title?
    review = review.split()
    review = [word for word in review if not word in stop_words_set]
    statements = ' '.join(review)
    sw_removed.append(statements)


In [ ]:
sw_removed_test = []

for i in range(len(test_data)):
    review = re.sub('[^a-zA-Z]',' ',test_data['text'][i])
    #not on title?
    review = review.split()
    review = [word for word in review if not word in stop_words_set]
    statements = ' '.join(review)
    sw_removed_test.append(statements)

In [ ]:
ps = PorterStemmer()

stemmed = []

print(len(sw_removed))

for i in range(len(sw_removed)):
  sw_removed_1 = sw_removed[i].split()
  stem = [ps.stem(word) for word in sw_removed_1]
  st = ' '.join(stem)
  stemmed.append(st)


18211


In [ ]:
stemmed_test = []

for i in range(len(sw_removed_test)):
  sw_removed_1 = sw_removed_test[i].split()
  stem = [ps.stem(word) for word in sw_removed_1]
  st = ' '.join(stem)
  stemmed_test.append(st)

In [ ]:
preprocess_arr = np.array(stemmed)
preprocess_df = pd.DataFrame(preprocess_arr)

In [ ]:
preprocess_arr_test = np.array(stemmed_test)
preprocess_df_test = pd.DataFrame(preprocess_arr_test)

In [ ]:
preprocess_df_test.head() 

,0
0,palo alto calif year scorn polit process silic...
1,video nodapl nativ american leader vow stay wi...
2,first succeed tri differ sport tim tebow heism...
3,min ago view comment like first time histori f...
4,sunday nbc meet press hous minor leader rep na...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


# vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
count_vectorizer = CountVectorizer(ngram_range=(1,3), max_features=3000)

# train_input = vectorizer.fit_transform(preprocess_arr).toarray()
train_input_count_vectorizer = count_vectorizer.fit_transform(preprocess_arr).toarray()

# transformed_df = pd.DataFrame(data=train_input, columns=vectorizer.get_feature_names())
transformed_df_count_vectorizer = pd.DataFrame(data=train_input_count_vectorizer, columns=count_vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
count_vectorizer_test = CountVectorizer(ngram_range=(1,3), max_features=3000)

# train_input = vectorizer.fit_transform(preprocess_arr).toarray()
test_input_count_vectorizer = count_vectorizer_test.fit_transform(preprocess_arr_test).toarray()

# transformed_df = pd.DataFrame(data=train_input, columns=vectorizer.get_feature_names())
transformed_df_count_vectorizer_test = pd.DataFrame(data=test_input_count_vectorizer, columns=count_vectorizer_test.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_test = transformed_df_count_vectorizer_test.values

K Fold

In [ ]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5)

# X = transformed_df.values
X_cv = transformed_df_count_vectorizer.values
y = train_data["label"].values 

# X_train=[]
# X_val=[]
X_train_cv=[]
X_val_cv=[]
y_train=[]
y_val=[]

for train_index, test_index in kf.split(X_cv,y):
  # X_train.append(X[train_index])
  # X_val.append(X[test_index]) 
  X_train_cv.append(X_cv[train_index])
  X_val_cv.append(X_cv[test_index]) 
  y_train.append(y[train_index])
  y_val.append(y[test_index])

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb_0 = GaussianNB()
gnb_1 = GaussianNB()
gnb_2 = GaussianNB()
gnb_3 = GaussianNB()
gnb_4 = GaussianNB()

gnb_0.fit(X_train_cv[0],y_train[0].astype('int'))
gnb_1.fit(X_train_cv[1],y_train[1].astype('int'))
gnb_2.fit(X_train_cv[2],y_train[2].astype('int'))
gnb_3.fit(X_train_cv[3],y_train[3].astype('int'))
gnb_4.fit(X_train_cv[4],y_train[4].astype('int'))

y_pred_0 = gnb_0.predict(X_val_cv[0])
y_pred_1 = gnb_1.predict(X_val_cv[1])
y_pred_2 = gnb_2.predict(X_val_cv[2])
y_pred_3 = gnb_3.predict(X_val_cv[3])
y_pred_4 = gnb_4.predict(X_val_cv[4])


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('Comparing the 5 Naive Bayes models on resp. val set')
#model 1
print('\nAverage of the evaluation Metrics for the Models:')
print("Accuracy: ", (accuracy_score(y_val[0].astype('int'), y_pred_0) + accuracy_score(y_val[1].astype('int'), y_pred_1) + accuracy_score(y_val[2].astype('int'), y_pred_2) + accuracy_score(y_val[3].astype('int'), y_pred_3) + accuracy_score(y_val[4].astype('int'), y_pred_4))/5)
print("Precision: ", (precision_score(y_val[0].astype('int'), y_pred_0) + precision_score(y_val[1].astype('int'), y_pred_1) + precision_score(y_val[2].astype('int'), y_pred_2) + precision_score(y_val[3].astype('int'), y_pred_3) + precision_score(y_val[4].astype('int'), y_pred_4))/5)
print("Recall: ", (recall_score(y_val[0].astype('int'), y_pred_0) + recall_score(y_val[1].astype('int'), y_pred_1) + recall_score(y_val[2].astype('int'), y_pred_2) + recall_score(y_val[3].astype('int'), y_pred_3) + recall_score(y_val[4].astype('int'), y_pred_4))/5)
print("f1-score: ", (f1_score(y_val[0].astype('int'), y_pred_0) + f1_score(y_val[1].astype('int'), y_pred_1) + f1_score(y_val[2].astype('int'), y_pred_2) + f1_score(y_val[3].astype('int'), y_pred_3) + f1_score(y_val[4].astype('int'), y_pred_4))/5)


In [ ]:
len(y_test)

In [ ]:
print('Comparing the Naive Bayes model on the test set')

gnb_test = GaussianNB()

gnb_test.fit(X_cv,y.astype(int))

y_test_pred = gnb_test.predict(X_test)


print('\nEvaluation Metrics for the test set:')
print("Accuracy: ", accuracy_score(y_test.astype('int'), y_test_pred))
print("Precision: ", precision_score(y_test.astype('int'), y_test_pred))
print("Recall: ", recall_score(y_test.astype('int'), y_test_pred))
print("f1-score: ", f1_score(y_test.astype('int'), y_test_pred))